In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection as ms
from sklearn import preprocessing
import sklearn.decomposition as skde
import sklearn.metrics as sklm
import sklearn.datasets as skds
import graphviz 
from sklearn import tree
import itertools

%matplotlib inline

In [2]:
bikesData = pd.read_csv('BikesProcessed.csv')
Labels = np.array(bikesData['BikeBuyer'])
bikesData.drop(bikesData.columns[0], axis=1, inplace=True)
bikesData.drop(['BikeBuyer'], axis=1, inplace=True)
Features = np.array(bikesData)
print(Features.shape)
bikesData.head()

(16404, 12)


,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,1,0,0,500,1,1,0,0,1,3,0,1
1,0,1,3,500,1,0,0,0,1,3,1,1
2,1,1,3,500,1,1,0,0,1,2,1,1
3,0,1,0,500,0,0,0,0,1,2,0,0
4,1,4,5,500,0,0,0,0,1,2,0,1


In [3]:
important_features = pd.read_csv('Top_CV_DecisionTree.csv')
important_features.head()

,Unnamed: 0,Unnamed: 0.1,size,accuracy,combs
0,2388,2388,6,0.793000,"('NumberChildrenAtHome', 'Sex', 'Married', 'In..."
1,3242,3242,7,0.792400,"('NumberChildrenAtHome', 'Sex', 'Married', 'Jo..."
2,1403,1403,5,0.792375,"('NumberChildrenAtHome', 'Sex', 'Married', 'Ag..."
3,2313,2313,6,0.792125,"('NumberChildrenAtHome', 'AreaCode', 'Sex', 'M..."
4,1401,1401,5,0.792125,"('NumberChildrenAtHome', 'Sex', 'Married', 'In..."


In [4]:
combs = np.array(important_features['combs'])
feature_combs = []
for item in combs:
    item = item.replace('(', '')
    item = item.replace(')', '')
    item = item.replace(',', '')
    item = item.replace('\'', '')
    item = item.split()
    feature_combs.append(item)
feature_combs

[['NumberChildrenAtHome',
  'Sex',
  'Married',
  'Income',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'Sex',
  'Married',
  'JobType',
  'EducationType',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome', 'Sex', 'Married', 'AgeBracket', 'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'AreaCode',
  'Sex',
  'Married',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome', 'Sex', 'Married', 'Income', 'AgeBracket'],
 ['HomeOwnerFlag',
  'NumberChildrenAtHome',
  'Sex',
  'Married',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'Sex',
  'Married',
  'EducationType',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome',
  'AreaCode',
  'Sex',
  'Married',
  'JobType',
  'EducationType',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberChildrenAtHome', 'Sex', 'Married', 'JobType', 'AgeBracket'],
 ['HomeOwnerFlag',
  'NumberChildrenAtHome',
  'Sex',
  'Married',
  'Income',
  'AgeBracket',
  'ChildrenFlag'],
 ['NumberCarsOwned',
  'NumberCh

In [5]:
#Scale the data
scaler = preprocessing.StandardScaler().fit(Features)
Features = scaler.transform(Features)
titles = list(bikesData)
bikes_scaled = pd.DataFrame()
count = 0
Features = pd.DataFrame(Features)

for title in titles:
    bikes_scaled[title] = Features[count]
    count+=1

bikes_scaled.head()

/Users/grahamcooper/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/grahamcooper/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,0.695306,-1.316615,-0.654523,-1.024189,0.989328,0.923656,-1.128274,-1.148595,-0.296932,1.246941,-1.011425,0.590773
1,-1.438215,-0.438729,1.326679,-1.024189,0.989328,-1.082654,-1.128274,-1.148595,-0.296932,1.246941,0.288843,0.590773
2,0.695306,-0.438729,1.326679,-1.024189,0.989328,0.923656,-1.128274,-1.148595,-0.296932,0.398228,0.288843,0.590773
3,-1.438215,-0.438729,-0.654523,-1.024189,-1.010787,-1.082654,-1.128274,-1.148595,-0.296932,0.398228,-1.011425,-1.692697
4,0.695306,2.194929,2.647481,-1.024189,-1.010787,-1.082654,-1.128274,-1.148595,-0.296932,0.398228,-1.011425,0.590773


In [6]:
def score_model(probs, threshold):
    return np.array([1 if x > threshold else 0 for x in probs[:,1]])

In [7]:
#cv = ms.ShuffleSplit(n_splits=8, test_size=5000, random_state=0)
Labels = Labels.reshape(Labels.shape[0],)
#scoring = ['accuracy','precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']

In [8]:
size = []
accuracy = []
#f1 = []
#precision = []
#recall = []
#auc = []
est_max_features = []
est_max_depth = []
cv_mean = []
cv_std = []
count = 0
for comb in feature_combs:
    comb_features = np.array(bikes_scaled[np.array(comb)])
    
    nr.seed(123)
    inside = ms.KFold(n_splits=10, shuffle = True)
    nr.seed(321)
    outside = ms.KFold(n_splits=10, shuffle = True)
    
    ## Define the dictionary for the grid search and the model object to search on
    param_grid = {"max_features": range(1, len(comb)+1), "max_depth": range(1,16)}
    ## Define the random forest model
    nr.seed(3456)
    rf_clf = DecisionTreeClassifier()

    ## Perform the grid search over the parameters
    nr.seed(4455)
    rf_clf = ms.GridSearchCV(estimator = rf_clf, param_grid = param_grid, 
                          cv = inside, # Use the inside folds
                          scoring = 'accuracy', #scoring,
                          return_train_score = True)
    rf_clf.fit(comb_features, Labels)
    est_max_features.append(rf_clf.best_estimator_.max_features)
    est_max_depth.append(rf_clf.best_estimator_.max_depth)
    
    nr.seed(498)
    cv_estimate = ms.cross_val_score(rf_clf, comb_features, Labels, 
                                 cv = outside) # Use the outside folds
    cv_mean.append(np.mean(cv_estimate))
    cv_std.append(np.std(cv_estimate))

    # Randomly sample cases to create independent training and test data

    nr.seed(1115)
    indx = range(comb_features.shape[0])
    indx = ms.train_test_split(indx, test_size = 5000)
    X_train = comb_features[indx[0],:]
    y_train = np.ravel(Labels[indx[0]])
    X_test = comb_features[indx[1],:]
    y_test = np.ravel(Labels[indx[1]])
    
    nr.seed(1115)
    rf_mod = DecisionTreeClassifier(max_features = rf_clf.best_estimator_.max_features, 
                                    max_depth = rf_clf.best_estimator_.max_depth) 
    rf_mod.fit(X_train, y_train)
    
    probabilities = rf_mod.predict_proba(X_test)
    scores = score_model(probabilities, 0.75)
    
    accuracy.append(sklm.accuracy_score(y_test, scores))
    #f1.append(np.mean(scores['test_f1_macro']))
    #precision.append(np.mean(scores['test_precision_macro']))
    #recall.append(np.mean(scores['test_recall_macro']))
    #auc.append(np.mean(scores['test_roc_auc']))
    size.append(len(comb))
    count += 1
    print(round(count/len(feature_combs), 2))

0.02
0.04
0.05
0.07
0.09
0.11
0.13
0.15
0.16
0.18
0.2
0.22
0.24
0.25
0.27
0.29
0.31
0.33
0.35
0.36
0.38
0.4
0.42
0.44
0.45
0.47
0.49
0.51
0.53
0.55
0.56
0.58
0.6
0.62
0.64
0.65
0.67
0.69
0.71
0.73
0.75
0.76
0.78
0.8
0.82
0.84
0.85
0.87
0.89
0.91
0.93
0.95
0.96
0.98
1.0


In [9]:
midx = pd.DataFrame()
midx['est_max_features'] = est_max_features
midx['est_max_depth'] = est_max_depth
midx['cv_mean'] = cv_mean
midx['cv_std'] = cv_std
midx['size'] = size
midx['CV_accuracy'] = important_features['accuracy']
midx['accuracy'] = accuracy
#midx['precision'] = precision
#midx['recall'] = recall
#midx['f1'] = f1
#midx['AUC'] = auc
midx['combs'] = feature_combs
midx.to_csv('NCV_DecisionTree2.csv')